In [1]:
using MixedLayerModel
using FileIO
using DifferentialEquations
using Sundials
using Revise
using Plots

using Logging: global_logger
using TerminalLoggers: TerminalLogger
global_logger(TerminalLogger());

In [25]:
include("mlm_solve_funcs.jl")

# set-up basic 400 ppm simulation
par = basic_params();
par.etype = bflux();
path = "./output/bflux_timeevolve/";

# run to steady-state
u0, sol = run_mlm(par);

# get output
code = sol.retcode;
uf = sol.u[end];
du = zeros(4);
mlm(du, uf, par, 0.0);
zi,hM,qM,SST = uf;
zb = calc_LCL(zi,hM,qM);
println(code);
println(uf);
println(du ./ uf);

# save to file
output = Dict("code" => code, "p"=>par, "u0" => u0, "uf" => uf, "du/u" => du./uf, 
"we" => we(uf,par,par.etype), "zb" => zb, "zc" => zi-zb,
"RHsurf" => RH(0.0, hM, qM), "LHF" => calc_LHF(uf,par), "SHF" => calc_SHF(uf,par),
"ΔR" => calc_cloudtop_RAD(uf,par,par.rtype), "OHU" => calc_OHU(uf,par,par.stype));
save(path*"co2_400.jld2", output)

# run steady-state
u0, solss = run_mlm_ss(par);
uf = solss.u;
println(uf);
# plot
t = sol.t / 3600.0 / 24.0
zi = getindex.(sol.u,1)
hM = getindex.(sol.u,2) * 1e-3
qtM = getindex.(sol.u,3) * 1e3
plot(t, [zi, hM, qtM], marker="o-", label=["zi(t) [m]" "hM(t) [kJ/kg]" "qtM(t) [g/kg]"], legend=:top, layout=(3,1))
hline!([uf[1]], label="SS, "*string(round(uf[1],digits=1))*" [m]", subplot=1)
hline!([uf[2]*1e-3], label="SS, "*string(round(uf[2]*1e-3,digits=1))*" [kJ/kg]", subplot=2)
hline!([uf[3]*1e3], label="SS, "*string(round(uf[3]*1e3,digits=2))*" [g/kg]", subplot=3)
title!(string(newCO2)*" (ppm)", subplot=1)
xaxis!("t (days)", subplot=3)
mkpath(replace(path, "output" => "figures"))
savefig(replace(path, "output" => "figures")*"co2_400.png")

ODE   0%|                                               |  ETA: N/A
ODE  21%|█████████▊                                     |  ETA: 0:00:03
ODE  42%|███████████████████▋                           |  ETA: 0:00:02
ODE  62%|█████████████████████████████▍                 |  ETA: 0:00:01
ODE  83%|███████████████████████████████████████▏       |  ETA: 0:00:01


  3.249631 seconds (23.85 M allocations: 744.442 MiB, 2.20% gc time)
Success
[914.0844994859599, 312455.74758193997, 0.008769258507783524, 290.0]
[-8.660179815163548e-10, -2.6027569727870523e-12, 6.916097854286773e-11, 0.0]
euler, dt=4 hrs

ODE 100%|███████████████████████████████████████████████| Time: 0:00:03


ODE   0%|                                               |  ETA: N/A
ODE  62%|█████████████████████████████▍                 |  ETA: 0:00:01


  3.309880 seconds (24.30 M allocations: 758.455 MiB, 2.12% gc time)
[914.0844994859599, 312455.74758193997, 0.008769258507783524, 290.0]


ODE 100%|███████████████████████████████████████████████| Time: 0:00:03
┌ Warning: Skipped marker arg o-.
└ @ Plots /Users/claresinger/.julia/packages/Plots/HcxwM/src/args.jl:1178


In [23]:
include("mlm_solve_funcs.jl")

# set co2
newCO2 = 600.0

# load initial condition from file
path = "./output/bflux_timeevolve/";
output = load(path*"co2_400.jld2");
u0 = output["uf"];
println(u0);
mlm(du, u0, par, 0.0);
println(du ./ u0);

# increase CO2, let SST evolve and check cloud changes
par.CO2 = newCO2;
u0, sol = run_mlm_from_init(u0, par);

# get output
code = sol.retcode;
uf = sol.u[end];
du = zeros(4);
mlm(du, uf, par, 0.0);
zi,hM,qM,SST = uf;
zb = calc_LCL(zi,hM,qM);
println(code);
println(uf);
println(du ./ uf);

# save to file
output = Dict("code" => code, "p" => par, "u0" => u0, "uf" => uf, "du/u" => du./uf, 
"we" => we(uf,par,par.etype), "zb" => zb, "zc" => zi-zb,
"RHsurf" => RH(0.0, hM, qM), "LHF" => calc_LHF(uf,par), "SHF" => calc_SHF(uf,par),
"ΔR" => calc_cloudtop_RAD(uf,par,par.rtype), "OHU" => calc_OHU(uf,par,par.stype))
save(path*"co2_upstep_fixSST_"*string(Int(newCO2))*".jld2", output)

# run steady-state
u0, solss = run_mlm_ss_from_init(u0, par);
uf = solss.u;
println(uf);

# plot
t = sol.t / 3600.0 / 24.0
zi = getindex.(sol.u,1)
hM = getindex.(sol.u,2) * 1e-3
qtM = getindex.(sol.u,3) * 1e3
plot(t, [zi, hM, qtM], marker="o-", label=["zi(t) [m]" "hM(t) [kJ/kg]" "qtM(t) [g/kg]"], legend=:top, layout=(3,1))
hline!([uf[1]], label="SS, "*string(round(uf[1],digits=1))*" [m]", subplot=1)
hline!([uf[2]*1e-3], label="SS, "*string(round(uf[2]*1e-3,digits=1))*" [kJ/kg]", subplot=2)
hline!([uf[3]*1e3], label="SS, "*string(round(uf[3]*1e3,digits=2))*" [g/kg]", subplot=3)
title!("fixed SST @ "*string(newCO2)*" (ppm)", subplot=1)
xaxis!("t (days)", subplot=3)
mkpath(replace(path, "output" => "figures"))
savefig(replace(path, "output" => "figures")*"co2_upstep_fixSST_"*string(Int(newCO2))*".png")

[914.0844994859599, 312455.74758193997, 0.008769258507783524, 290.0]
[-8.660179815163548e-10, -2.6027569727870523e-12, 6.916097854286773e-11, 0.0]


ODE   0%|                                               |  ETA: N/A
ODE  21%|█████████▊                                     |  ETA: 0:00:03
ODE  42%|███████████████████▋                           |  ETA: 0:00:02
ODE  62%|█████████████████████████████▍                 |  ETA: 0:00:01
ODE  83%|███████████████████████████████████████▏       |  ETA: 0:00:00


  2.970193 seconds (21.79 M allocations: 680.117 MiB, 2.37% gc time)
Success
[821.4687904807417, 313441.3067497841, 0.009081485160264555, 290.0]
[-4.275348380193225e-10, -1.5020002601886159e-12, 2.572859349004394e-11, 0.0]
euler

ODE 100%|███████████████████████████████████████████████| Time: 0:00:02


ODE   0%|                                               |  ETA: N/A
ODE  62%|█████████████████████████████▍                 |  ETA: 0:00:01


  2.726982 seconds (19.99 M allocations: 624.118 MiB, 2.11% gc time)
[821.5211577614712, 313441.3769412, 0.009081450318938342, 290.0]


ODE 100%|███████████████████████████████████████████████| Time: 0:00:02
┌ Warning: Skipped marker arg o-.
└ @ Plots /Users/claresinger/.julia/packages/Plots/HcxwM/src/args.jl:1178


In [24]:
include("mlm_solve_funcs.jl")

# set co2
newCO2 = 600.0

# load initial condition from file
path = "./output/bflux_timeevolve/";
output = load(path*"co2_400.jld2");
u0 = output["uf"];
OHU = output["OHU"];
println(u0);
mlm(du, u0, par, 0.0);
println(du ./ u0);

# set OHU, let SST evolve and check cloud changes
par.CO2 = newCO2;
par.Hw = 1;
par.OHU = OHU;
par.stype = varSST();
u0, sol = run_mlm_from_init(u0, par);

# get output
code = sol.retcode;
uf = sol.u[end];
du = zeros(4);
mlm(du, uf, par, 0.0);
zi,hM,qM,SST = uf;
zb = calc_LCL(zi,hM,qM);
println(code);
println(uf);
println(du ./ uf);

# save to file
output = Dict("code" => code, "p" => par, "u0" => u0, "uf" => uf, "du/u" => du./uf, 
"we" => we(uf,par,par.etype), "zb" => zb, "zc" => zi-zb,
"RHsurf" => RH(0.0, hM, qM), "LHF" => calc_LHF(uf,par), "SHF" => calc_SHF(uf,par),
"ΔR" => calc_cloudtop_RAD(uf,par,par.rtype), "OHU" => calc_OHU(uf,par,par.stype))
save(path*"co2_upstep_"*string(Int(newCO2))*".jld2", output)

# run steady-state
u0, solss = run_mlm_ss_from_init(u0, par);
uf = solss.u;
println(uf);

# plot
t = sol.t / 3600.0 / 24.0
zi = getindex.(sol.u,1)
hM = getindex.(sol.u,2) * 1e-3
qtM = getindex.(sol.u,3) * 1e3
plot(t, [zi, hM, qtM], marker="o-", label=["zi(t) [m]" "hM(t) [kJ/kg]" "qtM(t) [g/kg]"], legend=:top, layout=(3,1))
hline!([uf[1]], label="SS, "*string(round(uf[1],digits=1))*" [m]", subplot=1)
hline!([uf[2]*1e-3], label="SS, "*string(round(uf[2]*1e-3,digits=1))*" [kJ/kg]", subplot=2)
hline!([uf[3]*1e3], label="SS, "*string(round(uf[3]*1e3,digits=2))*" [g/kg]", subplot=3)
title!("slab-ocean @ "*string(newCO2)*" (ppm)", subplot=1)
xaxis!("t (days)", subplot=3)
mkpath(replace(path, "output" => "figures"))
savefig(replace(path, "output" => "figures")*"co2_upstep_"*string(Int(newCO2))*".png")

[914.0844994859599, 312455.74758193997, 0.008769258507783524, 290.0]
[-8.660179815163548e-10, 5.428217316186829e-8, 6.916097854286773e-11, 0.0]


ODE   0%|                                               |  ETA: N/A
ODE  21%|█████████▊                                     |  ETA: 0:00:04
ODE  42%|███████████████████▋                           |  ETA: 0:00:03
ODE  62%|█████████████████████████████▍                 |  ETA: 0:00:02
ODE  83%|███████████████████████████████████████▏       |  ETA: 0:00:01


  4.846668 seconds (35.75 M allocations: 1.090 GiB, 2.39% gc time)
Success
[1070.9056008487928, 317941.42375252757,

ODE 100%|███████████████████████████████████████████████| Time: 0:00:04


 0.00961182536250876, 292.9164637277062]
[6.132763478663549e-8, 2.5775426407372995e-9, 1.0536994537711904e-8, 1.0012844173540741e-9]
euler


ODE   0%|                                               |  ETA: N/A
ODE  62%|█████████████████████████████▍                 |  ETA: 0:00:02


  4.934900 seconds (36.45 M allocations: 1.111 GiB, 2.32% gc time)
[1070.9056008487928, 317941.42375252757, 0.00961182536250876, 292.9164637277062]


ODE 100%|███████████████████████████████████████████████| Time: 0:00:04
┌ Warning: Skipped marker arg o-.
└ @ Plots /Users/claresinger/.julia/packages/Plots/HcxwM/src/args.jl:1178
